### Decorator And Closures

In [1]:
def deco(func):
    print("Running deco()")
    def inner():
        print("Running inner()")
    return inner

@deco
def target():
    print("Running Target()")

Running deco()


In [2]:
target()

Running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
k = deco(target)

Running deco()


In [5]:
k()

Running inner()


### Variables Scope rules

In [6]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [7]:
b = 6
f1(3)

3
6


In [8]:
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [9]:
from dis import dis
dis(f1)

  1           RESUME                   0

  2           LOAD_GLOBAL              1 (print + NULL)
              LOAD_FAST                0 (a)
              CALL                     1
              POP_TOP

  3           LOAD_GLOBAL              1 (print + NULL)
              LOAD_GLOBAL              2 (b)
              CALL                     1
              POP_TOP
              RETURN_CONST             0 (None)


In [10]:
dis(f2)

  1           RESUME                   0

  2           LOAD_GLOBAL              1 (print + NULL)
              LOAD_FAST                0 (a)
              CALL                     1
              POP_TOP

  3           LOAD_GLOBAL              1 (print + NULL)
              LOAD_FAST_CHECK          1 (b)
              CALL                     1
              POP_TOP

  4           LOAD_CONST               1 (9)
              STORE_FAST               1 (b)
              RETURN_CONST             0 (None)


### Closures

Closures matter when you have nested functions. What matters is that it can access nonglobal variables that are defined outside of its body

In [11]:
class Averager:
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        return sum(self.series)/len(self.series)

In [12]:
avg = Averager()
avg(10)

10.0

In [13]:
avg(11)

10.5

In [14]:
avg(12)

11.0

In [22]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)  # here series is free-variable
        total = sum(series)
        return total/len(series)
    return averager

avg1 = make_averager()

In [23]:
avg1(10)

10.0

In [24]:
avg1(11)

10.5

In [25]:
avg1(12)

11.0

In [26]:
avg1.__code__.co_freevars

('series',)

In [27]:
avg1.__code__.co_varnames

('new_value', 'total')

In [28]:
avg1.__closure__

(<cell at 0x118d38df0: list object at 0x118b1bb00>,)

In [35]:
type(avg1.__closure__), len(avg1.__closure__)

(tuple, 1)

In [36]:
avg1.__closure__[0].cell_contents

[10, 11, 12]

### The nonlocal Declaration

In [37]:
def make_averager1():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [39]:
avg2 = make_averager1()

In [40]:
avg2(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [ ]:
# No free variables , due to assignment operation we implicitly creating local varibale count and total.
# therefore its no longer a free variable and there it is not saved in the closure
avg2.__code__.co_freevars

()

In [43]:
# fix is it use nonlocal keyword, its let you decleare free variable even when it is assigned within the function
def make_averager2():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager


In [44]:
avg3 = make_averager2()

In [45]:
avg3(10)

10.0

In [46]:
avg3(11)

10.5

In [47]:
avg3.__closure__

(<cell at 0x118d3d570: int object at 0x100e86988>,
 <cell at 0x118d3c7c0: int object at 0x100e86be8>)

In [51]:
avg3.__code__.co_freevars

('count', 'total')

# Implementing a Simple decorator

In [90]:
import time

def clock(func):
    """
    Record time elapsed by function.
    """
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        res = func(*args, **kwargs)
        t1 = time.perf_counter()
        elapsed = t1 - t0
        print(f"{elapsed: 0.8f} {func.__name__}{args}, kwargs: {kwargs} -> {res}")
        return res
    return clocked

In [91]:
@clock
def snooze(sec):
    time.sleep(sec)

In [79]:
snooze(3)

 3.00107354 snooze(3,), kwargs: {} -> None


In [92]:
@clock
def factorial(n):
    """
      calculate the n!
    """
    return 1 if n < 2 else factorial(n-1) * n

factorial(6)

 0.00000071 factorial(1,), kwargs: {} -> 1
 0.00006842 factorial(2,), kwargs: {} -> 2
 0.00008708 factorial(3,), kwargs: {} -> 6
 0.00010054 factorial(4,), kwargs: {} -> 24
 0.00011433 factorial(5,), kwargs: {} -> 120
 0.00012908 factorial(6,), kwargs: {} -> 720


720

In [93]:
factorial.__name__

'clocked'

In [94]:
factorial.__doc__

In [95]:
from functools import wraps

def clock(func):
    @wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        res = func(*args, **kwargs)
        t1 = time.perf_counter()
        elapsed = t1 - t0
        print(f"{elapsed: 0.8f} {func.__name__}{args}, kwargs: {kwargs} -> {res}")
        return res
    return clocked

In [96]:
import functools

In [98]:
@clock
def fibonaaci(n):
    return n if n < 2 else fibonaaci(n-2) + fibonaaci(n-1)

fibonaaci(6)

 0.00000033 fibonaaci(0,), kwargs: {} -> 0
 0.00000029 fibonaaci(1,), kwargs: {} -> 1
 0.00004950 fibonaaci(2,), kwargs: {} -> 1
 0.00000021 fibonaaci(1,), kwargs: {} -> 1
 0.00000017 fibonaaci(0,), kwargs: {} -> 0
 0.00000017 fibonaaci(1,), kwargs: {} -> 1
 0.00000958 fibonaaci(2,), kwargs: {} -> 1
 0.00001921 fibonaaci(3,), kwargs: {} -> 2
 0.00007887 fibonaaci(4,), kwargs: {} -> 3
 0.00000013 fibonaaci(1,), kwargs: {} -> 1
 0.00000017 fibonaaci(0,), kwargs: {} -> 0
 0.00000017 fibonaaci(1,), kwargs: {} -> 1
 0.00000942 fibonaaci(2,), kwargs: {} -> 1
 0.00001854 fibonaaci(3,), kwargs: {} -> 2
 0.00000017 fibonaaci(0,), kwargs: {} -> 0
 0.00000017 fibonaaci(1,), kwargs: {} -> 1
 0.00000967 fibonaaci(2,), kwargs: {} -> 1
 0.00000017 fibonaaci(1,), kwargs: {} -> 1
 0.00000021 fibonaaci(0,), kwargs: {} -> 0
 0.00000017 fibonaaci(1,), kwargs: {} -> 1
 0.00000971 fibonaaci(2,), kwargs: {} -> 1
 0.00001850 fibonaaci(3,), kwargs: {} -> 2
 0.00003696 fibonaaci(4,), kwargs: {} -> 3
 0.00006488

8

In [ ]:
# Faster implementation using cache
# here cache can consume all available memory, no bound to memory usage
@functools.cache
@clock
def fibonaaci(n):
    return n if n < 2 else fibonaaci(n-2) + fibonaaci(n-1)

fibonaaci(6)

 0.00000083 fibonaaci(0,), kwargs: {} -> 0
 0.00000083 fibonaaci(1,), kwargs: {} -> 1
 0.00009713 fibonaaci(2,), kwargs: {} -> 1
 0.00000142 fibonaaci(3,), kwargs: {} -> 2
 0.00012554 fibonaaci(4,), kwargs: {} -> 3
 0.00000092 fibonaaci(5,), kwargs: {} -> 5
 0.00015183 fibonaaci(6,), kwargs: {} -> 8


8

In [100]:
# lru_cacheis bounded by maxsize parameter, whihc means cache will hold max 128 recent entries.
@functools.lru_cache(maxsize=128)
@clock
def fibonaaci(n):
    return n if n < 2 else fibonaaci(n-2) + fibonaaci(n-1)

fibonaaci(6)

 0.00000071 fibonaaci(0,), kwargs: {} -> 0
 0.00000054 fibonaaci(1,), kwargs: {} -> 1
 0.00008992 fibonaaci(2,), kwargs: {} -> 1
 0.00000133 fibonaaci(3,), kwargs: {} -> 2
 0.00011808 fibonaaci(4,), kwargs: {} -> 3
 0.00000079 fibonaaci(5,), kwargs: {} -> 5
 0.00014508 fibonaaci(6,), kwargs: {} -> 8


8

### Singledispatched
<pre>
Python will automatically pick the correct version based on argument type.

⭐ How it works (core idea)

You define:
    one base function
    register implementations for different types</pre>

In [102]:
@functools.singledispatch
def serialize(obj):
    raise TypeError(f"Unsupported Type {type(obj)}")

@serialize.register
def _(obj: int):
    return f"int({obj})"

@serialize.register
def _(obj: str):
    return f"str('{obj}')"

@serialize.register
def _(obj: list):
    return "[" + ", ".join(serialize(x) for x in obj) + "]"


In [103]:
print(serialize(10))

int(10)


In [ ]:
print(serialize("hello"))

"str('hello')"

In [106]:
print(serialize(2.3))

TypeError: Unsupported Type <class 'float'>

In [ ]:
"""🔥 .dispatch(type)

Given a type, return the actual function that will be called for that type.

Not call it — just return the function object."""
serialize.dispatch(int)

<function __main__._(obj: int)>

In [109]:
"""
⭐ .registry -> This is the full internal registry mapping from types → function implementations.
"""

serialize.registry

mappingproxy({object: <function __main__.serialize(obj)>,
              int: <function __main__._(obj: int)>,
              str: <function __main__._(obj: str)>,
              list: <function __main__._(obj: list)>})

### Parametrized Decorators

In [119]:
registry = set()

def register(active=True):
    def decorate(func):
        print(f"running register active={active} -> decorate({func})")
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

In [120]:
@register(active=False)
def f1():
    print("running f1()")

@register()
def f2():
    print("running f2()")

def f3():
    print("running f3()")

running register active=False -> decorate(<function f1 at 0x107d214e0>)
running register active=True -> decorate(<function f2 at 0x107d21440>)


In [121]:
registry

{<function __main__.f2()>}

### Parameterized clock decorator


In [ ]:
DEFAULT_FMT = '[{elapsed: 0.8f}s] {name}({args}) -> {result}'
def clock_para(fmt=DEFAULT_FMT):
    def decorate(func):
        def clock(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clock
    return decorate

@clock_para()
def snooze(sec):
    time.sleep(sec)

snooze(2)

[ 2.00503162s] snooze(2) -> None 


In [133]:
@clock_para('{name}({args}) dt={elapsed:0.3f}s')
def snooze1(sec):
    time.sleep(sec)

snooze1(1)


snooze1(1) dt=1.002s


### A Class Based Clock Decorator

In [ ]:
DEFAULT_FMT = '[{elapsed: 0.8f}s] {name}({args}) -> {result}'

class Clock:
    def __init__(self, fmt = DEFAULT_FMT):
        self.fmt = fmt
    
    def __call__(self, func):
        def clocked(self, *_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked